# Use Case: Unity Software (U) analysis

The use case creates an agent that has the recent Unity Software (U) earnings reports and can anser questios about it.

We go through the following steps:
- create an agent
- list the agent to see the agent's configuration parameters
- upload a file to the agent knowledge-base
- add URLs to scrape the text from websites to knowledge base
- list the agent to see which documents the agent's index (knowledge base) includes
- interact with agent
- delete the agent

In [1]:
import os
import requests
from dotenv import load_dotenv
import sys

load_dotenv()

# this is just the test wrapper that makes agent output nice on this notebook

from textwrap import TextWrapper

def wrapped_print(text):
    wrapper = TextWrapper(width=150)
    # Replace newline characters with a placeholder
    text = text.replace('\n', '<newline_placeholder>')

    # Wrap the text
    wrapped_text = wrapper.fill(text)

    # Replace the placeholder with newline characters again
    formatted_text = wrapped_text.replace('<newline_placeholder>', '\n')

    print(formatted_text)


In [2]:
api_key = os.environ.get("BOTASTICO_API_KEY")
base_url = os.environ.get(f'BASE_URL')
headers = {'Authorization': f'Bearer {api_key}'}

In [3]:
base_url

'https://botastico-api-production-3qrwuust2a-nw.a.run.app'

# Create an agent

In [5]:
agent_data = {
        "agent_name": "Unity Software analyser",
        "source_text": "Unity Software Inc. operates a platform that provides real-time 3D development tools and services. Its platform provides software solutions to create, run, and monetize interactive, real-time 2D and 3D content for mobile phones, tablets, PCs, consoles, and augmented and virtual reality devices. The company serves enterprises, mid-market companies, and government and non-profit institutions; mid-sized, small, and independent businesses; and individuals across various industries. It offers its solutions directly through its online store, field sales operations, independent distributors, and resellers in North America, Denmark, China, Finland, the United Kingdom, Germany, Israel, Japan, Singapore, South Korea, Spain, and internationally. The company was founded in 2004 and is headquartered in San Francisco, California.",
        "source_urls": ["https://investors.unity.com/news/news-details/2023/Unity-Announces-Fourth-Quarter-and-Fiscal-Year-2022-Financial-Results-Webcast/default.aspx",
                        "https://investors.unity.com/news/news-details/2023/Unity-Announces-Fourth-Quarter-and-Full-Year-2022-Financial-Results/default.aspx"],
        "chunk_size": 400,
        "chunk_overlap": 0,
        "model_name": "gpt-3.5-turbo",
        "temperature": 0.5,
        "max_response_tokens": 500,
        "stop": "",
        "script": "",
        "assistant_name": "Unity Software analyser",
        "intro": {"en": "How can I help you?"},
        "sitelogo_url_or_svg": "https://s26.q4cdn.com/977690160/files/design/U_Logo_White_RGB.png",
        "url_loader": "unstructured",
    }

create_response = requests.post(f'{base_url}/v1/agents', json=agent_data, headers=headers)
print (create_response.status_code)
print(create_response.json())

200
{'agent_id': 'EWaLXDjjIV99IvB4NUTv', 'status': 'success'}


# List the agents

In [6]:
agent_id = create_response.json()['agent_id']
agent_id

'EWaLXDjjIV99IvB4NUTv'

In [ ]:
# 4.A. List agents after one agent is created
list_response = requests.get(f'{base_url}/v1/agents?agent_ids={agent_id}', headers=headers)
print (list_response.status_code)
list_response.json()

In [ ]:
# get from agents list an agent with agent_id as a key
[agent for agent in list_response.json() if agent['agent_id'] == agent_id][0]

# Upload files

In [ ]:
pdf_file_name = '2023_Q1-Shareholder-Letter_FINAL.pdf'
current_dir = os.getcwd()
test_pdf_path = os.path.join(current_dir, 'data_files', pdf_file_name)
with open(test_pdf_path, 'rb') as f:
    upload_response = requests.post(f'{base_url}/v1/agents/{agent_id}/kb/file', headers=headers, files={'file': (pdf_file_name, f)})

print (upload_response.status_code)
upload_response.json()

# Add URLs to scrape

In [ ]:
urls_to_scrape = ['https://investors.unity.com/news/news-details/2023/Unity-Named-to-Fast-Companys-Annual-List-of-the-Worlds-Most-Innovative-Companies-for-2023/default.aspx']
scrape_response = requests.post(f'{base_url}/v1/agents/{agent_id}/kb/urls/scrape', headers=headers, json={'urls': urls_to_scrape, 'loader_type': 'unstructured'})
print (scrape_response.status_code)
scrape_response.json()

# List agents

In [ ]:
# 4.A. List agents after one agent is created
list_response = requests.get(f'{base_url}/v1/agents', headers=headers)
print (list_response.status_code)
[agent for agent in list_response.json()]

# Interactions

In [20]:

# 8.A. Chat user sends a, interaction request to the agent
prompt = 'Make Unity SWOT analysis please' 
response = requests.post(f'{base_url}/v1/agents/{agent_id}/interactions', headers=headers, json={
    'prompt': prompt
})
print(response.status_code)
response_content = response.json()['choices'][0]['message']['content']
wrapped_print(response_content)

200
Sure, here's a SWOT analysis for Unity:

Strengths:
1. Market Leader: Unity is one of the
leading platforms for creating and operating interactive 3D content. It has a strong presence in the gaming industry and is also expanding into other
industries.
2. Robust Platform: Unity offers a comprehensive set of software solutions that support creators through the entire
development lifecycle. This makes it a one-stop-shop for developers.
3. Large User Base: Unity's platform is used by millions of
developers around the world, creating a strong network effect.
4. Strong Partnerships: Unity has formed strong partnerships with
major companies such as Outfit7, Second Dinner, and more. These partnerships help to boost Unity's reputation and user
base.

Weaknesses:
1. High Dependence on Gaming Industry: While Unity is expanding into
other industries, it still heavily relies on the gaming industry. Any downturn in this industry could significantly impact Unity's
performance.
2. Workforce Reduct

# Add history in interactions

In [22]:
history = [{"role": "user", "content": prompt}, {"role": "assistant", "content": response_content}]

prompt = 'What is the recent quater results impact to the SWOT analysis you made?' 
response = requests.post(f'{base_url}/v1/agents/{agent_id}/interactions', headers=headers, json={
    'prompt': prompt, 'history': history
})
print(response.status_code)
response_content = response.json()['choices'][0]['message']['content']
wrapped_print(response_content)


200
Apologies for the misunderstanding earlier. Let's revisit the SWOT analysis in light of Unity's recent quarterly
results.

Strengths:
- Strong Revenue Growth: The 56% year-over-year increase in revenue
reaffirms Unity's strong market position and the effectiveness of its business model. This could enhance its reputation and attract more developers to
its platform.
- Profitability Improvement: The improved adjusted EBITDA indicates that Unity is moving towards profitability,
which strengthens its financial position.

Weaknesses:
- Continued Net Loss: Despite
strong revenue growth, Unity reported a GAAP net loss of $254 million. This indicates that the company is still struggling to control costs and
achieve consistent profitability.

Opportunities:
- AI Adoption: Unity’s excitement about
the rate of adoption of AI tools, and their expectation that AI will be a tailwind for many years to come, suggests a significant opportunity. The
company's ability to leverage AI could drive furthe

# Delete the agent

In [ ]:

response = requests.delete(f'{base_url}/v1/agents/{agent_id}', headers=headers)
print(response.status_code)

response.json()['message']